### 1. 載入所需套件

In [1]:
# 安裝套件
!pip install -q groq aisuite[all] newsapi-python ccxt yfinance ta matplotlib gradio pycoingecko

In [2]:
# 導入套件
import os               # 檔案操作套件
import re               # 正則表達式(regular expression)套件
import json              # JSON 處理套件
from google.colab import userdata  # Google Colab 用戶資料套件
import aisuite as ai         # LLM 開發使用套件
from groq import Groq        # Groq 套件
from newsapi import NewsApiClient  # NewsAPI 套件(提供來自全球超過150,000 個來源的最新和歷史新聞文章的JSON 搜索結果。)
import ccxt              # 加密貨幣交易所套件
import yfinance as yf        # Yahoo Finance 套件
import ta               # 技術分析套件
import time              # 時間套件
import matplotlib.pyplot as plt   # 繪圖套件
import gradio as gr         # Gradio UI套件
import pandas as pd         # 數據處理套件
from datetime import datetime, timedelta  # 日期時間套件
from pycoingecko import CoinGeckoAPI     # 加密貨幣API套件
from matplotlib.font_manager import FontProperties as font # 載入文字字型

### 2. 讀入你的金鑰

請依你使用的服務, *決定讀入哪個金鑰*

In [3]:
#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY'] = api_key
provider = "groq"
model1 = "llama3-70b-8192"      # 擅長處理長上下文處理的任務
model2 = "llama-3.3-70b-versatile"  # 擅長處理處理多種類型的輸入

建立Multiagent Collaboration，讓多個AI Agent 扮演不同角色，彼此合作完成任務。

In [4]:
# Prompt Analyzer Agent（解析使用者輸入、提取使用者喜好跟想詢問的資訊）
provider_prompt_analyzer = "groq"
model_prompt_analyzer = model1

# 基本面分析Agent(分析該標的物的基本面)
provider_basic_analyzer = "groq"
model_basic_analyzer = model1

# 技術面分析Agent(分析該標的物的技術面)
provider_tech_analyzer = "groq"
model_tech_analyzer = model1

# 新聞情緒分析Agent(分析該標的物的消息面)
provider_news_analyzer = "groq"
model_news_analyzer = model2

# 策略生成Agent（Writer）
provider_writer = "groq"
model_writer = model1

# 策略審查Agent（Reviewer）
provider_reviewer = "groq"
model_reviewer = model2

### 3. 定義reply函數

In [5]:
def reply(system="請用台灣習慣的中文回覆所有的內容，若是其他語言也請把所有內容翻譯成中文。",
          prompt=None,
          provider="groq",
          model=model1):
    try:
        client = ai.Client()

        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ]

        # 動態選擇模型用於不同任務(model1, model2)
        if model not in [model1, model2]:
            raise ValueError(f"不支援的模型：{model}，請使用 {model1} 或 {model2}")

        response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)
        content = response.choices[0].message.content
        print(f"API 回應：{content}")
        return content
    except Exception as e:
        error_msg = f"API 呼叫失敗：{str(e)}"
        print(error_msg)
        return error_msg

### 4. 設計所有AI Agent(Multiagent Collaboration)

#### 4-1. Prompt Analyzer Agent（解析使用者輸入）

In [6]:
system_prompt_analyzer = """
你是一個專業的prompt解析Agent，負責分析使用者的輸入，並提取以下資訊：
1. 股票代碼（必須是正確真實的代碼，例如2330.TW、TSLA、NVDA、BTC/USDT、SOL/USDT）。
2. 投資偏好（例如技術面、籌碼面、新聞時事、綜合分析）。
3. 投資週期（極短線、短線、中線、長線）。
4. 其他要求（例如投資組合比例分配、風險控制、報酬率計算、技術指標區間）。
如果使用者未提供某些資訊，請使用以下預設值：
- 投資偏好：綜合分析（基本面、技術面、新聞時事）。
- 投資週期：中線。
請以純JSON格式輸出解析結果，不要包含任何額外文字或說明，例如：
{
  "ticker": "2330.TW",
  "preference": "技術面",
  "investment_horizon": "短線",
  "other_requirements": "RSI區間需低於30"
}
"""
# 定義函數-前處理，將使用者輸入加入prompt，以便後續分析。
def parse_user_input(user_input, model=model1):
    prompt = f"""
    使用者輸入：{user_input}
    請解析上述輸入，提取股票代碼、投資偏好、投資週期和其他要求，並以JSON格式輸出。
    """
    response = reply(system=system_prompt_analyzer,
                     prompt=prompt,
                     provider=provider_prompt_analyzer,
                     model=model)
    if "API 呼叫失敗" in response:
        raise ValueError(response)

    # 清理回應，提取純 JSON 部分
    json_match = re.search(r'\{.*\}', response, re.DOTALL)
    if json_match:
        json_str = json_match.group(0)
        try:
            parsed_response = json.loads(json_str)
            return parsed_response
        except json.JSONDecodeError as e:
            raise ValueError(f"無法解析提取的 JSON 部分：{json_str}, 錯誤：{str(e)}")
    else:
        raise ValueError(f"無法從 API 回應中提取 JSON：{response}")

#### 4-2. 資料搜集Agent（Tool Use）

In [7]:
# 初始化NewsAPI
newsapi = NewsApiClient(api_key='前往 NewsAPI 官網註冊或更新一個有效的 API Key。')


# 定義蒐集即時新聞的函式
def fetch_real_time_news(ticker, investment_horizon):
    try:
        # 今天日期
        end_date = datetime.now()  # 2025-04-21
        # NewsAPI 免費版限制只能查詢 30 天前的資料
        max_allowed_date = end_date - timedelta(days=25)  # 2025-03-20

        # 根據投資期限選擇查詢範圍，但不超過 NewsAPI 限制
        if investment_horizon == "ultra_short":
            days_back = 3
        elif investment_horizon == "short":
            days_back = 7
        elif investment_horizon == "medium":
            days_back = 15
        else:  # long
            days_back = 25

        # 計算開始日期，但不得早於 max_allowed_date
        to_date = end_date  # 查詢到今天
        start_date = to_date - timedelta(days=days_back)
        if start_date < max_allowed_date:
            start_date = max_allowed_date  # 確保不早於 2025-03-20

        # 確保日期範圍有效
        if start_date > to_date:
            start_date, to_date = to_date, start_date

        query = ticker.split('.')[0]  # 移除 .TW 這類相關後綴
        # 抓取相關的所有中文文章
        articles = newsapi.get_everything(
            q=query,
            language='zh',
            sort_by='relevancy',
            from_param=start_date.strftime('%Y-%m-%d'),
            to=to_date.strftime('%Y-%m-%d')
        )

        if articles['status'] != 'ok':
            return f"新聞 API 錯誤：{articles.get('message', '未知錯誤')}"

        top_articles = articles['articles'][:20]
        news_texts = [article['content'] for article in top_articles if article['content']]
        if not news_texts:
            return "未找到相關新聞資料。"

        return news_texts
    except Exception as e:
        return f"新聞搜集失敗：{str(e)}"

# 定義蒐集加密貨幣數據的函式
def fetch_crypto_data(symbol):
    try:
        exchange = ccxt.binance()
        ticker = exchange.fetch_ticker(symbol)
        return ticker
    except Exception as e:
        return {"error": f"加密貨幣資料獲取失敗：{str(e)}"}


# 定義備用加密貨幣數據獲取函數（使用 CoinGecko）
def fetch_crypto_data_from_coingecko(symbol, days):
    """
    使用 CoinGecko API 獲取加密貨幣數據。

    Args:
        symbol: 加密貨幣符號（例如 'BTC/USDT'）
        days: 數據範圍（天數）

    Returns:
        包含歷史數據的 DataFrame 或錯誤訊息
    """
    cg = CoinGeckoAPI()
    try:
        # 將符號轉換為 CoinGecko 的 ID（例如 'BTC' -> 'bitcoin'）
        coin_id = symbol.split('/')[0].lower()
        coin_id_map = {
            'btc': 'bitcoin',
            'eth': 'ethereum',
            'sol': 'solana',
            # 可根據需要添加更多映射
        }
        coin_id = coin_id_map.get(coin_id, coin_id)

        # 獲取歷史數據
        data = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency='usd', days=days)
        prices = data['prices']
        df = pd.DataFrame(prices, columns=['timestamp', 'price'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)

        # 模擬 yfinance 的歷史數據格式
        df['Open'] = df['price']
        df['High'] = df['price']
        df['Low'] = df['price']
        df['Close'] = df['price']
        df['Volume'] = 0  # CoinGecko 不提供成交量，設為 0
        return df
    except Exception as e:
        return {"error": f"CoinGecko 資料獲取失敗：{str(e)}"}

# 定義獲取加密貨幣數據的函數（整合 Yahoo Finance 和 CoinGecko）
def fetch_crypto_data(symbol, investment_horizon):
    """
    獲取加密貨幣數據，先嘗試 Yahoo Finance，若失敗則使用 CoinGecko。

    Args:
        symbol: 加密貨幣符號（例如 'BTC/USDT'）
        investment_horizon: 投資期限（用於確定數據範圍）

    Returns:
        歷史數據 DataFrame 或錯誤訊息
    """
    # 將符號轉換為 Yahoo Finance 格式（例如 'BTC/USDT' -> 'BTC-USD'）
    ticker = symbol.replace('/USDT', '-USD')

    # 根據投資期限選擇數據範圍
    period_map = {
        'ultra_short': '1mo',
        'short': '3mo',
        'medium': '1y',
        'long': '2y'
    }
    period = period_map.get(investment_horizon, '1y')

    # 根據 period 計算天數（用於 CoinGecko）
    days_map = {
        '1mo': 30,
        '3mo': 90,
        '1y': 365,
        '2y': 730
    }
    days = days_map.get(period, 365)

    # 嘗試使用 Yahoo Finance 獲取數據
    max_retries = 3
    for attempt in range(max_retries):
        try:
            stock = yf.Ticker(ticker)
            history = stock.history(period=period)
            if history.empty:
                raise Exception("歷史數據為空")
            return history
        except Exception as e:
            if attempt == max_retries - 1:  # 最後一次重試仍失敗
                break
            time.sleep(2)  # 等待 2 秒後重試

    # 若 Yahoo Finance 失敗，嘗試使用 CoinGecko
    crypto_data = fetch_crypto_data_from_coingecko(symbol, days)
    if isinstance(crypto_data, dict) and "error" in crypto_data:
        return crypto_data
    return crypto_data

# 定義蒐集股市數據的函式
def fetch_stock_data(parsed_input):
    ticker = parsed_input['ticker']
    investment_horizon = parsed_input.get('investment_horizon', 'medium')

    # 根據投資期限選擇歷史數據的時間範圍
    period_map = {
        'ultra_short': '1mo',  # 極短線：1 個月
        'short': '3mo',        # 短線：3 個月
        'medium': '1y',        # 中線：1 年
        'long': '2y'           # 長線：2 年
    }
    period = period_map.get(investment_horizon, '1y')

    # 修正加密貨幣代碼格式
    is_crypto = False
    original_ticker = ticker
    if '/' in ticker:
        is_crypto = True
        ticker = ticker.replace('/USDT', '-USD')  # 將 BTC/USDT 轉換為 BTC-USD
    elif ticker.endswith('-USD'):
        is_crypto = True
        ticker = ticker  # 已經是 BTC-USD 格式，保持不變
    else:
        is_crypto = False

    # 獲取股票基本資訊和歷史數據
    try:
        if is_crypto:
            # 對於加密貨幣，調用 fetch_crypto_data
            history = fetch_crypto_data(original_ticker, investment_horizon)
            if isinstance(history, dict) and "error" in history:
                return history
            info = {"symbol": ticker}  # 簡單模擬 info（加密貨幣可能沒有完整的 info）
        else:
            # 對於普通股票，使用 yf.Ticker
            stock = yf.Ticker(ticker)
            info = stock.info
            history = stock.history(period=period)
            if history.empty:
                return {"error": f"股票 {ticker} 的歷史數據為空"}
    except Exception as e:
        return {"error": f"無法獲取股票資料：{str(e)}"}

    # 計算技術指標（始終計算短期和長期移動平均線）
    try:
        df = history.copy()
        # 始終計算短期和中期移動平均線，以便繪圖使用
        ma_periods = [20, 50]  # 至少計算 SMA_20 和 SMA_50
        if investment_horizon == 'ultra_short':
            ma_periods.extend([1, 5])
        elif investment_horizon == 'short':
            ma_periods.extend([5, 10])
        elif investment_horizon == 'long':
            ma_periods.extend([100, 200])

        for period in ma_periods:
            df[f'SMA_{period}'] = ta.trend.sma_indicator(df['Close'], window=period)
            df[f'EMA_{period}'] = ta.trend.ema_indicator(df['Close'], window=period)
            df[f'WMA_{period}'] = ta.trend.wma_indicator(df['Close'], window=period)

        if investment_horizon == 'long':
            for period in [100, 200]:
                df[f'CMA_{period}'] = df['Close'].expanding(min_periods=period).mean()

        df['RSI_14'] = ta.momentum.rsi(df['Close'], window=14)
        df['MACD'] = ta.trend.macd(df['Close'])
        df['MACD_Signal'] = ta.trend.macd_signal(df['Close'])

        bb_window = 20 if investment_horizon == 'medium' else (5 if investment_horizon == 'ultra_short' else (10 if investment_horizon == 'short' else 50))
        df['BB_upper'] = ta.volatility.bollinger_hband(df['Close'], window=bb_window)
        df['BB_middle'] = ta.volatility.bollinger_mavg(df['Close'], window=bb_window)
        df['BB_lower'] = ta.volatility.bollinger_lband(df['Close'], window=bb_window)

    except Exception as e:
        return {"error": f"技術指標計算失敗：{str(e)}"}

    # 獲取新聞數據
    news = None
    try:
        news = fetch_real_time_news(ticker, investment_horizon)
    except Exception as e:
        news = {"warning": f"新聞搜集失敗：{str(e)}"}

    # 獲取加密貨幣數據
    crypto_data = None

    return {
        "basic_info": info,
        "history": history,
        "tech_indicators": df,
        "news": news,
        "crypto_data": crypto_data,
        "investment_horizon": investment_horizon
    }

#### 4-3. 基本面分析Agent

In [8]:
system_basic_analyzer = """
請用台灣習慣的中文回覆所有的內容文字，若是其他語言也請把所有內容翻譯成中文。你是一個專業的基本面分析師，專注於分析公司的財報和基本面資料
，利用財務分析和經濟學上的研究來評估企業價值或預測證券價值的走勢。內容包含一家公司的財務報表和非財務上的資訊，如商品需求成長性的預測、企業比較、
新制度的影響分析或人口的改變等。請逐步推理，給出具體正確的分析結論。請全部的字都用台灣習慣的中文回覆，並將所有內容翻譯成中文，包含數字和專有名詞的適當轉換。
"""

def analyze_basic_data(data, ticker):
    if "error" in data:
        return data["error"]
    prompt = f"""
    以下是股票 {ticker} 的基本面資料：
    {data['basic_info']}
    請分析這些資料，給出結論。
    """
    return reply(system=system_basic_analyzer,
                 prompt=prompt,
                 provider=provider_basic_analyzer,
                 model=model_basic_analyzer)

#### 4-4. 技術面分析Agent

In [9]:
system_tech_analyzer = """
請用台灣習慣的中文回覆所有的內容，若是其他語言也請把所有內容翻譯成中文。你是一個專業的技術面分析師，專注於技術指標
（例如RSI、MACD、移動平均線、SMA、EMA、WMA、CMA等），分析過去價格走勢並預測未來價格走勢的市場價格分析方法，利用市場此種相關規律以尋找與過去價格走勢類似的模式。
請根據使用者的偏好，逐步推理，給出分析結論。請用台灣習慣的中文回覆，並將所有內容翻譯成中文，包含數字和專有名詞的適當轉換。
"""

# 計算技術指標
def calculate_technical_indicators(history, investment_horizon='medium'):
    # 複製歷史股價數據
    df = history.copy()

    # 根據投資期限選擇均線的時間範圍
    if investment_horizon == 'ultra_short':
        ma_periods = [1, 5]  # 極短線均線
    elif investment_horizon == 'short':
        ma_periods = [5, 10]  # 短線均線
    elif investment_horizon == 'medium':
        ma_periods = [20, 50]  # 中線均線
    else:  # long
        ma_periods = [100, 200]  # 長線均線

    # 計算移動平均線
    for period in ma_periods:
        df[f'SMA_{period}'] = ta.trend.sma_indicator(df['Close'], window=period)  # 簡單移動平均線
        df[f'EMA_{period}'] = ta.trend.ema_indicator(df['Close'], window=period)  # 指數移動平均線
        df[f'WMA_{period}'] = ta.trend.wma_indicator(df['Close'], window=period)  # 加權移動平均線

    # 計算 CMA（累積移動平均線，通常用於長線分析）
    if investment_horizon == 'long':
        for period in [100, 200]:
            df[f'CMA_{period}'] = df['Close'].expanding(min_periods=period).mean()  # 累積移動平均線

    # 其他技術指標
    df['RSI_14'] = ta.momentum.rsi(df['Close'], window=14)  # 14 日 RSI
    df['MACD'] = ta.trend.macd(df['Close'])  # MACD
    df['MACD_Signal'] = ta.trend.macd_signal(df['Close'])  # MACD 訊號線

    # 布林通道（根據投資期限調整窗口）
    bb_window = 20 if investment_horizon == 'medium' else (5 if investment_horizon == 'ultra_short' else (10 if investment_horizon == 'short' else 50))
    df['BB_upper'] = ta.volatility.bollinger_hband(df['Close'], window=bb_window)  # 布林通道上軌
    df['BB_middle'] = ta.volatility.bollinger_mavg(df['Close'], window=bb_window)  # 布林通道中軌
    df['BB_lower'] = ta.volatility.bollinger_lband(df['Close'], window=bb_window)  # 布林通道下軌

    return df

# 技術面分析
def analyze_tech_data(data, parsed_input):
    if "error" in data:
        return data["error"]

    ticker = parsed_input['ticker']
    preference = parsed_input['preference']
    other_requirements = parsed_input['other_requirements']
    investment_horizon = parsed_input.get('investment_horizon', 'medium')  # 預設為中線

    # 根據投資期限選擇要顯示的技術指標
    if investment_horizon == 'ultra_short':
        indicators = ['SMA_1', 'EMA_1', 'WMA_1', 'SMA_5', 'EMA_5', 'WMA_5', 'RSI_14', 'MACD', 'MACD_Signal', 'BB_upper', 'BB_middle', 'BB_lower']
    elif investment_horizon == 'short':
        indicators = ['SMA_5', 'EMA_5', 'WMA_5', 'SMA_10', 'EMA_10', 'WMA_10', 'RSI_14', 'MACD', 'MACD_Signal', 'BB_upper', 'BB_middle', 'BB_lower']
    elif investment_horizon == 'medium':
        indicators = ['SMA_20', 'EMA_20', 'WMA_20', 'SMA_50', 'EMA_50', 'WMA_50', 'RSI_14', 'MACD', 'MACD_Signal', 'BB_upper', 'BB_middle', 'BB_lower']
    else:  # long
        indicators = ['SMA_20', 'EMA_20', 'WMA_20', 'SMA_100', 'EMA_100', 'WMA_100', 'SMA_200', 'EMA_200', 'WMA_200', 'CMA_100', 'CMA_200', 'RSI_14', 'MACD', 'MACD_Signal', 'BB_upper', 'BB_middle', 'BB_lower']

    # 確保只選擇 DataFrame 中存在的欄位
    available_indicators = [indicator for indicator in indicators if indicator in data['tech_indicators'].columns]

    # 提取最近一筆技術指標數據
    tech_data = data['tech_indicators'][available_indicators].tail(1)

    # 更新提示詞，確保prompt要求符合需求
    prompt = f"""
    以下是股票 {ticker} 的技術面資料：
    投資期限：{investment_horizon}
    技術指標（最近一筆數據）：{tech_data.to_dict()}
    使用者偏好：{preference}
    其他要求：{other_requirements}

    請用台灣習慣的中文回覆所有的內容，若是其他語言也請把所有內容翻譯成中文。請根據上述資料和使用者要求，進行技術面分析，並給出結論。回覆必須以台灣慣用的中文為主，所有分析和描述
    必須使用中文，但專有名詞（如 RSI、MACD、Bollinger Bands）可以保留英文。禁止整句或整段使用英文，英文只能少量出現在專有名詞中，並確保回覆清晰易懂，技術指標需明確指明計算週期（例如「50 日移動平均線」）。
    """
    return reply(system=system_tech_analyzer,
                 prompt=prompt,
                 provider=provider_tech_analyzer,
                 model=model_tech_analyzer)

#### 4-5. 新聞分析Agent

In [10]:
system_news_analyzer = """
請用台灣習慣的中文回覆所有的內容，若是其他語言也請把所有內容翻譯成中文。你是一個專業的新聞情緒分析師，分析新聞的情緒（正面、負面、中性），
並評估對股票價格的影響。請逐步推理，給出分析結論。請用台灣習慣的中文回覆，並將所有內容翻譯成中文，包含數字和專有名詞的適當轉換。
"""

# 定義分析新聞的函式
def analyze_news_data(data, parsed_input):
    if "error" in data:
        return data["error"]
    if isinstance(data['news'], str):  # 如果新聞資料是錯誤訊息，直接返回
        return data['news']
    ticker = parsed_input['ticker']
    preference = parsed_input['preference']

    prompt = f"""
    以下是股票 {ticker} 的新聞資料：
    {data['news']}
    使用者偏好：{preference}
    請分析新聞情緒，並評估對股票價格的影響。
    """
    return reply(system=system_news_analyzer,
                 prompt=prompt,
                 provider=provider_news_analyzer,
                 model=model_news_analyzer)

#### 4-6. 策略生成與優化Agent（Writer與Reviewer）

In [11]:
system_writer = """
請用台灣習慣的中文回覆所有的內容，若是其他語言也請把所有內容翻譯成中文。你是一個專業的股票策略制定者，更是一個歷經無數投資市場的佼佼者，跟巴菲特一樣優秀。
根據給定的分析結果，撰寫一段具體明確的投資策略跟建議，使使用者可以很好地獲得優渥報酬。請用台灣習慣的中文回覆，並將所有內容翻譯成中文，包含數字和專有名詞的適當轉換。
"""

system_reviewer = """
請用台灣習慣的中文回覆所有的內容，若是其他語言也請把所有內容翻譯成中文。你是一個專業的策略審查者，負責審查投資策略並提出改進建議，讓策略更具體、更實用、更容易執行。
請用台灣習慣的中文回覆，並將所有內容翻譯成中文，包含數字和專有名詞的適當轉換。
"""

# 定義產生跟優化策略的函式
def generate_and_refine_strategy(basic_analysis, tech_analysis, news_analysis, parsed_input):
    ticker = parsed_input['ticker']
    preference = parsed_input['preference']
    investment_horizon = parsed_input['investment_horizon']
    other_requirements = parsed_input['other_requirements']

    # 初步策略
    first_prompt = f"""
    以下是對股票 {ticker} 的分析：
    基本面分析：{basic_analysis}
    技術面分析：{tech_analysis}
    新聞情緒分析：{news_analysis}
    使用者偏好：{preference}
    投資週期：{investment_horizon}
    其他要求：{other_requirements}
    請根據這些分析，撰寫一段投資策略建議（約300個中文字）。
    """
    first_strategy = reply(system=system_writer, prompt=first_prompt)

    # 進行3層Reflection
    current_strategy = first_strategy
    suggestions = []
    for i in range(3):
        suggestion_prompt = f"""
        以下是目前的投資策略：
        {current_strategy}
        # 請審查這段策略，並提出改進建議，讓它更具體、更實用。內容以400字以內，請不要超過字數限制。並且確保使用台灣習慣的中文回覆所有的內容，若是其他語言也請把所有內容翻譯成中文。
        """
        suggestion = reply(system=system_reviewer, prompt=suggestion_prompt)
        suggestions.append(suggestion)

        next_prompt = f"""
        以下是我撰寫的策略：
        {current_strategy}
        以下是審查者的建議：
        {suggestion}
        請根據這些建議，改進這段策略，讓它更具體、更實用。內容以400字以內，請不要超過字數限制。並且確保使用台灣習慣的中文回覆所有的內容，若是其他語言也請把所有內容翻譯成中文。
        """
        current_strategy = reply(system=system_writer, prompt=next_prompt)

    # 提取技術指標數據以計算具體價格
    data = fetch_stock_data(parsed_input)
    if "error" in data:
        return first_strategy, suggestions, f"最終策略生成失敗：{data['error']}"

    history = data['history']
    tech_indicators = data['tech_indicators']
    current_price = history['Close'].iloc[-1]
    latest_rsi = tech_indicators['RSI_14'].iloc[-1]
    latest_sma_200 = tech_indicators.get('SMA_200', tech_indicators['SMA_50']).iloc[-1]  # 如果沒有 SMA_200，用 SMA_50 代替
    latest_macd = tech_indicators['MACD'].iloc[-1]
    latest_macd_signal = tech_indicators['MACD_Signal'].iloc[-1]

    # 計算 RSI_14 = 70 時的價格
    close_prices = history['Close'].tail(14).copy()
    if len(close_prices) < 14:
        return first_strategy, suggestions, "最終策略生成失敗：歷史數據不足"

    deltas = close_prices.diff()
    gains = deltas.where(deltas > 0, 0).mean()
    losses = -deltas.where(deltas < 0, 0).mean()
    if losses == 0:
        losses = 1e-10
    avg_gains = gains * 13
    avg_losses = losses * 13
    target_rs_70 = 7 / 3
    y_70 = (target_rs_70 * avg_losses - avg_gains) / 13
    price_at_rsi_70 = current_price + y_70 if y_70 > 0 else current_price * 1.10

    # 假設 MACD 買入信號出現在價格突破 SMA_200 時
    macd_buy_price = latest_sma_200

    # 改進最終策略，加入具體價格
    final_strategy = current_strategy
    final_strategy += f"\n\n**具體價格建議**：\n"
    final_strategy += f"- **買入點**：當價格突破 200 日移動平均線（{latest_sma_200:.2f} 元）且 MACD 顯示買入信號（MACD 上穿 MACD_Signal）時買入。\n"
    final_strategy += f"- **賣出點**：當價格上漲至 {price_at_rsi_70:.2f} 元（估計 RSI_14 達到 70）時賣出。\n"
    final_strategy += f"- **停損點**：若價格下跌 8% 至 {current_price * 0.92:.2f} 元，建議賣出以控制風險。\n"

    return first_strategy, suggestions, final_strategy

#### 4-7. 簡單明瞭投資策略

In [12]:
# 定義簡單投資策略函數
def generate_simple_strategy(data):
    if "error" in data:
        return data["error"]

    history = data['history']
    tech_indicators = data['tech_indicators']

    # 獲取最新價格和技術指標
    current_price = history['Close'].iloc[-1] if not history.empty else "無法獲取"
    latest_rsi = tech_indicators['RSI_14'].iloc[-1] if 'RSI_14' in tech_indicators else "無法獲取"
    latest_sma_20 = tech_indicators['SMA_20'].iloc[-1] if 'SMA_20' in tech_indicators else "無法獲取"
    latest_sma_50 = tech_indicators['SMA_50'].iloc[-1] if 'SMA_50' in tech_indicators else "無法獲取"
    latest_macd = tech_indicators['MACD'].iloc[-1] if 'MACD' in tech_indicators else "無法獲取"
    latest_macd_signal = tech_indicators['MACD_Signal'].iloc[-1] if 'MACD_Signal' in tech_indicators else "無法獲取"

    # 檢查是否為無效數據
    if any(isinstance(x, str) for x in [current_price, latest_rsi, latest_sma_20, latest_sma_50, latest_macd, latest_macd_signal]):
        return "無法生成簡單策略：技術指標或價格數據缺失"

    # 判斷 MACD 趨勢
    macd_bullish = latest_macd > latest_macd_signal  # MACD 上穿 MACD_Signal，看漲
    macd_bearish = latest_macd < latest_macd_signal  # MACD 下穿 MACD_Signal，看跌
    macd_diff = latest_macd - latest_macd_signal  # MACD 與 Signal 的差值

    # 計算 RSI_14 = 30 和 RSI_14 = 70 時的價格（簡化估計）
    # 獲取過去 14 天的收盤價
    close_prices = history['Close'].tail(14).copy()
    if len(close_prices) < 14:
        return "無法計算 RSI 價格：歷史數據不足"

    # 計算當前 RSI 的基礎數據
    deltas = close_prices.diff()
    gains = deltas.where(deltas > 0, 0).mean()
    losses = -deltas.where(deltas < 0, 0).mean()
    if losses == 0:
        losses = 1e-10  # 避免除以 0

    # 假設最後一天價格變化，估計 RSI_14 = 30 和 RSI_14 = 70 時的價格
    # RSI = 100 - 100 / (1 + RS)，RS = 平均上漲幅度 / 平均下跌幅度
    # RSI = 30 時，RS = 3/7 ≈ 0.4286
    # RSI = 70 時，RS = 7/3 ≈ 2.3333
    avg_gains = gains * 13  # 過去 13 天的總上漲
    avg_losses = losses * 13  # 過去 13 天的總下跌

    # 估計 RSI_14 = 30 時的價格（假設下跌）
    target_rs_30 = 3 / 7
    # 假設最後一天下跌 X，則 (avg_gains + 0) / (avg_losses + X) = 3/7
    # X = (avg_gains - 3/7 * avg_losses) / (3/7 * 13)
    x_30 = (avg_gains - target_rs_30 * avg_losses) / (target_rs_30 * 13)
    price_at_rsi_30 = current_price - x_30 if x_30 > 0 else current_price * 0.95  # 假設下跌 5% 作為預估

    # 估計 RSI_14 = 70 時的價格（假設上漲）
    target_rs_70 = 7 / 3
    # 假設最後一天上漲 Y，則 (avg_gains + Y) / (avg_losses + 0) = 7/3
    # Y = (7/3 * avg_losses - avg_gains) / 13
    y_70 = (target_rs_70 * avg_losses - avg_gains) / 13
    price_at_rsi_70 = current_price + y_70 if y_70 > 0 else current_price * 1.10  # 假設上漲 10% 作為預估

    # 定義簡單策略（基於技術指標）
    strategy = f"**技術投資策略（結合價值與趨勢）**\n\n"
    strategy += f"**目前價格**：{current_price:.2f} 元\n"
    strategy += f"**技術指標概覽**：\n- RSI_14：{latest_rsi:.2f}\n- SMA_20：{latest_sma_20:.2f}\n- SMA_50：{latest_sma_50:.2f}\n- MACD：{latest_macd:.2f}，MACD_Signal：{latest_macd_signal:.2f}\n\n"

    # 進場條件：RSI 超賣 + MACD 看漲
    if latest_rsi < 30 and macd_bullish:
        entry_price = current_price
        target_price = latest_sma_20  # 假設反彈到 SMA_20
        strategy += f"**進場價格**：\n- 現在可以進場！建議以 {entry_price:.2f} 元進場。\n- 目標價格：{target_price:.2f} 元（反彈至 20 日移動平均線）。\n- 原因：RSI_14（{latest_rsi:.2f}）顯示超賣，且 MACD 看漲（MACD 上穿 MACD_Signal，差值：{macd_diff:.2f}）。\n\n"
    else:
        strategy += f"**進場價格**：\n- 當價格下跌至 {price_at_rsi_30:.2f} 元（估計 RSI_14 達到 30）且 MACD 上穿 MACD_Signal 時進場。\n- 目前 RSI_14：{latest_rsi:.2f}，MACD 趨勢：{'看漲' if macd_bullish else '看跌'}（MACD 與 Signal 差值：{macd_diff:.2f}）。\n\n"

    # 出場條件：RSI 超買 + MACD 看跌
    if latest_rsi > 70 and macd_bearish:
        exit_price = current_price
        strategy += f"**出場價格**：\n- 現在可以出場！建議以 {exit_price:.2f} 元賣出。\n- 原因：RSI_14（{latest_rsi:.2f}）顯示超買，且 MACD 看跌（MACD 下穿 MACD_Signal，差值：{macd_diff:.2f}）。\n\n"
    else:
        strategy += f"**出場價格**：\n- 當價格上漲至 {price_at_rsi_70:.2f} 元（估計 RSI_14 達到 70）且 MACD 下穿 MACD_Signal 時賣出。\n- 目前 RSI_14：{latest_rsi:.2f}，MACD 趨勢：{'看漲' if macd_bullish else '看跌'}（MACD 與 Signal 差值：{macd_diff:.2f}）。\n\n"

    # 加碼條件：價格突破 SMA_20 + MACD 看漲
    if current_price > latest_sma_20 and macd_bullish:
        add_price = current_price
        target_price = latest_sma_50  # 假設趨勢持續到 SMA_50
        strategy += f"**加碼價格**：\n- 現在可以加碼！建議以 {add_price:.2f} 元加碼。\n- 目標價格：{target_price:.2f} 元（趨勢可能持續至 50 日移動平均線）。\n- 原因：價格突破 20 日移動平均線（{latest_sma_20:.2f}），且 MACD 看漲（MACD 與 Signal 差值：{macd_diff:.2f}）。\n\n"
    else:
        add_price = latest_sma_20
        strategy += f"**加碼價格**：\n- 當價格突破 {add_price:.2f} 元（20 日移動平均線）且 MACD 看漲時加碼。\n- 目前價格：{current_price:.2f} 元，MACD 趨勢：{'看漲' if macd_bullish else '看跌'}（MACD 與 Signal 差值：{macd_diff:.2f}）。\n\n"

    # 減碼條件：價格跌破 SMA_20 + MACD 看跌
    if current_price < latest_sma_20 and macd_bearish:
        reduce_price = current_price
        strategy += f"**減碼價格**：\n- 現在建議減碼！以 {reduce_price:.2f} 元減少持倉。\n- 原因：價格跌破 20 日移動平均線（{latest_sma_20:.2f}），且 MACD 看跌（MACD 下穿 MACD_Signal，差值：{macd_diff:.2f}）。\n\n"
    else:
        reduce_price = latest_sma_20
        strategy += f"**減碼價格**：\n- 當價格跌破 {reduce_price:.2f} 元（20 日移動平均線）且 MACD 看跌時減碼。\n- 目前價格：{current_price:.2f} 元，MACD 趨勢：{'看漲' if macd_bullish else '看跌'}（MACD 與 Signal 差值：{macd_diff:.2f}）。\n\n"

    strategy += "這個策略結合技術指標和趨勢分析，適合追求穩健投資的人，但請注意市場波動，並根據個人風險承受能力調整操作。"

    return strategy

#### 4-8. stock_strategy_app

In [13]:
# 繪製該股票的圖表
def plot_stock_data(history, tech_indicators, investment_horizon='medium'):
    plt.figure(figsize=(20, 12))
    plt.plot(history['Close'], label='收盤價')

    # 根據投資期限選擇適當的移動平均線
    if investment_horizon == 'ultra_short':
        short_ma = 'SMA_1'
        long_ma = 'EMA_5'
    elif investment_horizon == 'short':
        short_ma = 'SMA_5'
        long_ma = 'EMA_10'
    elif investment_horizon == 'medium':
        short_ma = 'SMA_20'
        long_ma = 'EMA_50'
    else:  # long
        short_ma = 'SMA_20'  # 即使是長期，也繪製短期移動平均線
        long_ma = 'EMA_100'

    # 確保欄位存在再繪製
    if short_ma in tech_indicators.columns:
        plt.plot(tech_indicators[short_ma], label=f'{short_ma}')
    if long_ma in tech_indicators.columns:
        plt.plot(tech_indicators[long_ma], label=f'{long_ma}')

    plt.title('Stock Price Chart')
    plt.legend()
    plt.savefig('stock_plot.png')
    plt.close()
    return 'stock_plot.png'

# Gradio 介面
def stock_strategy_app(user_input):
    try:
        parsed_input = parse_user_input(user_input)
    except Exception as e:
        return f"輸入解析失敗：{str(e)}", "", "", "", "", "", "", ""

    data = fetch_stock_data(parsed_input)
    if "error" in data:
        return data["error"], "", "", "", "", "", "", ""

    basic_analysis = analyze_basic_data(data, parsed_input['ticker'])
    tech_analysis = analyze_tech_data(data, parsed_input)
    news_analysis = analyze_news_data(data, parsed_input)

    first_strategy, suggestions, final_strategy = generate_and_refine_strategy(
        basic_analysis, tech_analysis, news_analysis, parsed_input
    )

    # 生成簡單投資策略
    simple_strategy = generate_simple_strategy(data)

    # 格式化 suggestions，讓每個建議分隔並加上編號
    formatted_suggestions = "\n\n".join(
        f"改進建議 {i+1}：\n{suggestion}" for i, suggestion in enumerate(suggestions)
    )

    try:
        plot_path = plot_stock_data(data['history'], data['tech_indicators'], data['investment_horizon'])
    except Exception as e:
        plot_path = None
        final_strategy += f"\n\n⚠️ 圖表生成失敗：{str(e)}"

    return user_input, basic_analysis, tech_analysis, news_analysis, first_strategy, formatted_suggestions, final_strategy, simple_strategy, plot_path

#### 4-9. 測試完整效果是否都順利應用

In [14]:
# 測試程式碼
def test_agents():
    print("=== 開始測試 AI Agents ===")

    test_input = "我想分析台積電2330.TW，偏好技術面，短線操作，RSI低於30時進場"

# 測試 1: Prompt Analyzer Agent
    print("\n測試 1: Prompt Analyzer Agent")
    parsed_input = None
    try:
        print("使用 model1 測試...")
        parsed_input = parse_user_input(test_input, model=model1)
        print("解析結果:", parsed_input)
        assert "ticker" in parsed_input, "解析結果缺少 ticker"
        assert "preference" in parsed_input, "解析結果缺少 preference"
        assert "investment_horizon" in parsed_input, "解析結果缺少 investment_horizon"
        print("提示解析成功！")
    except Exception as e:
        print(f"提示解析失敗（model1）：{str(e)}")
        print("改用 model2 測試...")
        try:
            parsed_input = parse_user_input(test_input, model=model2)
            print("解析結果:", parsed_input)
            assert "ticker" in parsed_input, "解析結果缺少 ticker"
            assert "preference" in parsed_input, "解析結果缺少 preference"
            assert "investment_horizon" in parsed_input, "解析結果缺少 investment_horizon"
            print("提示解析成功！")
        except Exception as e:
            print(f"提示解析失敗（model2）：{str(e)}")
            return

# 測試 2: 資料搜集 Agent
    print("\n測試 2: 資料搜集 Agent")
    data = None
    try:
        data = fetch_stock_data(parsed_input)
        print("資料搜集結果:", {k: str(v)[:100] + "..." if isinstance(v, (dict, list, pd.DataFrame)) else v for k, v in data.items()})
        assert "basic_info" in data, "缺少基本面資料"
        assert "history" in data, "缺少歷史價格資料"
        assert "tech_indicators" in data, "缺少技術指標資料"
        assert "news" in data, "缺少新聞資料"
        print("資料搜集成功！")
    except Exception as e:
        print(f"資料搜集失敗：{str(e)}")
        return

# 測試 3: 基本面分析 Agent
    print("\n測試 3: 基本面分析 Agent")
    basic_analysis = None
    try:
        basic_analysis = analyze_basic_data(data, parsed_input['ticker'])
        print("基本面分析結果:", basic_analysis[:100] + "..." if len(basic_analysis) > 100 else basic_analysis)
        assert isinstance(basic_analysis, str), "基本面分析結果應為字串"
        assert len(basic_analysis) > 0, "基本面分析結果為空"
        print("基本面分析成功！")
    except Exception as e:
        print(f"基本面分析失敗：{str(e)}")
        return

# 測試 4: 技術面分析 Agent
    print("\n測試 4: 技術面分析 Agent")
    tech_analysis = None
    try:
        tech_analysis = analyze_tech_data(data, parsed_input)
        print("技術面分析結果:", tech_analysis[:100] + "..." if len(tech_analysis) > 100 else tech_analysis)
        assert isinstance(tech_analysis, str), "技術面分析結果應為字串"
        assert len(tech_analysis) > 0, "技術面分析結果為空"
        print("技術面分析成功！")
    except Exception as e:
        print(f"技術面分析失敗：{str(e)}")
        return

# 測試 5: 新聞情緒分析 Agent
    print("\n測試 5: 新聞情緒分析 Agent")
    news_analysis = None
    try:
        news_analysis = analyze_news_data(data, parsed_input)
        print("新聞情緒分析結果:", news_analysis[:100] + "..." if len(news_analysis) > 100 else news_analysis)
        assert isinstance(news_analysis, str), "新聞情緒分析結果應為字串"
        assert len(news_analysis) > 0, "新聞情緒分析結果為空"
        print("新聞情緒分析成功！")
    except Exception as e:
        print(f"新聞情緒分析失敗：{str(e)}")
        return

# 測試 6: 策略生成與優化 Agent
    print("\n測試 6: 策略生成與優化 Agent")
    try:
        first_strategy, suggestions, final_strategy = generate_and_refine_strategy(
            basic_analysis, tech_analysis, news_analysis, parsed_input
        )
        print("初步策略:", first_strategy[:100] + "..." if len(first_strategy) > 100 else first_strategy)
        print("改進建議:", [s[:100] + "..." if len(s) > 100 else s for s in suggestions])
        print("最終策略:", final_strategy[:100] + "..." if len(final_strategy) > 100 else final_strategy)
        assert isinstance(first_strategy, str), "初步策略應為字串"
        assert len(suggestions) == 3, "應有 3 個改進建議"
        assert isinstance(final_strategy, str), "最終策略應為字串"
        print("策略生成與優化成功！")
    except Exception as e:
        print(f"策略生成與優化失敗：{str(e)}")
        return

    print("\n=== 測試完成 ===")

# 執行測試
test_agents()

=== 開始測試 AI Agents ===

測試 1: Prompt Analyzer Agent
使用 model1 測試...
API 回應：{
  "ticker": "2330.TW",
  "preference": "技術面",
  "investment_horizon": "短線",
  "other_requirements": "RSI區間需低於30"
}
解析結果: {'ticker': '2330.TW', 'preference': '技術面', 'investment_horizon': '短線', 'other_requirements': 'RSI區間需低於30'}
提示解析成功！

測試 2: 資料搜集 Agent
資料搜集結果: {'basic_info': "{'address1': 'Hsinchu Science Park', 'address2': 'No. 8, Li-Hsin Road 6', 'city': 'Hsinchu City', 'z...", 'history': '                                 Open        High         Low       Close  \\\nDate                   ...', 'tech_indicators': '                                 Open        High         Low       Close  \\\nDate                   ...', 'news': "['2330 3 2,859.57 2 10.0% 46.5% 3 8,392.54 41.6%', 'Sony PS5 3 3 15 2 NintendoSwitch\\r\\nFamitsu 2025...", 'crypto_data': None, 'investment_horizon': '短線'}
資料搜集成功！

測試 3: 基本面分析 Agent
API 回應：Based on the provided data, here is a comprehensive analysis of Taiwan Semiconductor Manufa

### 5. 用 Gradio 打造你的對話機器人 Web App!

In [15]:
# Gradio 介面
with gr.Blocks() as demo:
    # 插入標題與描述設定
    gr.Markdown("# 📈 股票策略 AI Agent")
    gr.Markdown(
        """
        <div class="markdown-description">請輸入您的需求，可輸入台股、美股跟加密貨幣喔!加入股票代碼，如: 2330.TW、BTCUSDT、NVDA會更能正確生成!</div>
        """
    )

    with gr.Row():
        user_input = gr.Textbox(label="請輸入您的需求", placeholder="例如：我想分析台積電2330.TW，偏好技術面，短線操作，RSI低於30時進場")
        submit_btn = gr.Button("生成分析與策略", variant="primary")

    with gr.Tabs():
        with gr.Tab(label="策略建議"):
            with gr.Row():
                with gr.Column(scale=5):
                    gr.Markdown("### 📝 核心策略建議", elem_classes="large-text")
                    simple_strategy_output = gr.Textbox(label="簡單具體投資策略（適合初學者）", interactive=False, elem_classes="large-text")
                    final_strategy_output = gr.Textbox(label="最終策略", interactive=False, elem_classes="large-text")
                with gr.Column(scale=5):
                    gr.Markdown("### 📊 價格走勢與技術指標圖", elem_classes="large-text")
                    plot_output = gr.Image(label="價格走勢與技術指標圖", elem_classes="large-image")

        with gr.Tab(label="詳細分析"):
            with gr.Row():
                with gr.Column(scale=3):
                    gr.Markdown("### 📊 分析結果（參考資訊）", elem_classes="medium-text")
                    user_input_display = gr.Textbox(label="您的需求", interactive=False, elem_classes="user-input-text", lines=2)
                    basic_output = gr.Textbox(label="基本面分析", interactive=False, elem_classes="basic-output-text", lines=8)
                    tech_output = gr.Textbox(label="技術面分析", interactive=False, elem_classes="tech-output-text", lines=8)
                    news_output = gr.Textbox(label="新聞情緒分析", interactive=False, elem_classes="news-output-text", lines=8)
                with gr.Column(scale=3):
                    gr.Markdown("### 📝 策略生成過程", elem_classes="medium-text")
                    first_strategy_output = gr.Textbox(label="初步策略", interactive=False, elem_classes="first-strategy-text", lines=12)
                    suggestions_output = gr.Textbox(label="改進建議", interactive=False, elem_classes="suggestions-text", lines=12)

    demo.css = """
    .large-text, .large-text label, .large-text textarea {
        font-size: 14px !important;
        color: #ffffff;
        background-color: #1e2a44;
        padding: 8px;
        border-radius: 5px;
        margin-bottom: 10px;
    }
    .medium-text, .medium-text label, .medium-text textarea {
        font-size: 14px !important;
        color: #d1d5db;
        background-color: #2d3748;
        padding: 6px;
        border-radius: 5px;
        margin-bottom: 8px;
    }
    .small-text, .small-text label, .small-text textarea {
        font-size: 14px !important;
        color: #9ca3af;
        background-color: #4a5568;
        padding: 4px;
        border-radius: 5px;
        margin-bottom: 5px;
    }
    .user-input-text, .user-input-text label, .user-input-text textarea {
        font-size: 14px !important;
        color: #d1d5db;
        background-color: #2d3748;  /* 深藍灰 */
        padding: 6px;
        border-radius: 5px;
        margin-bottom: 8px;
    }
    .basic-output-text, .basic-output-text label, .basic-output-text textarea {
        font-size: 14px !important;
        color: #a3e4d7;  /* 淺綠 */
        background-color: #1a3c34;  /* 深綠 */
        padding: 6px;
        border-radius: 5px;
        margin-bottom: 8px;
    }
    .tech-output-text, .tech-output-text label, .tech-output-text textarea {
        font-size: 14px !important;
        color: #d1c4e9;  /* 淺紫 */
        background-color: #3c2f47;  /* 深紫 */
        padding: 6px;
        border-radius: 5px;
        margin-bottom: 8px;
    }
    .news-output-text, .news-output-text label, .news-output-text textarea {
        font-size: 14px !important;
        color: #f4c7c3;  /* 淺紅 */
        background-color: #4a2c2a;  /* 深棕 */
        padding: 6px;
        border-radius: 5px;
        margin-bottom: 8px;
    }
    .first-strategy-text, .first-strategy-text label, .first-strategy-text textarea {
        font-size: 14px !important;
        color: #b0c4de;  /* 淺藍 */
        background-color: #2e4053;  /* 深藍 */
        padding: 6px;
        border-radius: 5px;
        margin-bottom: 8px;
    }
    .suggestions-text, .suggestions-text label, .suggestions-text textarea {
        font-size: 14px !important;
        color: #b0c4de;  /* 淺藍 */
        background-color: #34495e;  /* 稍淺的深藍 */
        padding: 6px;
        border-radius: 5px;
        margin-bottom: 8px;
    }
    .large-image, .large-image img {
        width: 100% !important;
        height: 300px !important;
        object-fit: contain;
    }
    .gr-button-primary {
        background-color: #f97316;
        color: white;
        border: none;
        padding: 10px 20px;
        border-radius: 5px;
    }
    .gr-button-primary:hover {
        background-color: #ea580c;
    }
    """

    submit_btn.click(
        stock_strategy_app,
        inputs=[user_input],
        outputs=[
            user_input_display,
            basic_output,
            tech_output,
            news_output,
            first_strategy_output,
            suggestions_output,
            final_strategy_output,
            simple_strategy_output,
            plot_output
        ]
    )

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://247a9cf8a961b3afab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


API 回應：{
  "ticker": "2317.TW",
  "preference": "綜合分析",
  "investment_horizon": "中線",
  "other_requirements": ""
}
API 回應：After analyzing the provided financial data of Hon Hai Precision Industry Co., Ltd. (stock code: 2317.TW), I will provide a comprehensive conclusion on the company's performance, valuation, and prospects.

**Company Overview**

Hon Hai Precision Industry Co., Ltd. is a Taiwan-based company primarily engaged in the provision of electronic OEM services. The company offers a diverse range of products, including consumer electronics, cloud networking products, desktop computers, notebook computers, tablets, and automotive electronic parts. With operations in multiple countries, Hon Hai Precision Industry Co., Ltd. is a leading player in the technology sector.

**Financial Performance**

Based on the provided data, here are some key observations:

1. **Revenue Growth**: The company has demonstrated a moderate revenue growth rate of 0.15, indicating a steady increase in r

<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 25910 (\N{CJK UNIFIED IDEOGRAPH-6536}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')
<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 30436 (\N{CJK UNIFIED IDEOGRAPH-76E4}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')
<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 20729 (\N{CJK UNIFIED IDEOGRAPH-50F9}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')


API 回應：{
  "ticker": "DOT/USDT",
  "preference": "綜合分析",
  "investment_horizon": "長線",
  "other_requirements": "策略建議"
}
API 回應：Based on the provided fundamental data, I will conduct a thorough analysis to provide a comprehensive conclusion.

**Company Overview**
The symbol DOT-USD represents Polkadot (DOT), a decentralized platform that enables the interoperability of multiple blockchain networks. As a cryptocurrency, DOT is the native token of the Polkadot network.

**Financial Statements**
Unfortunately, as a cryptocurrency, Polkadot does not have traditional financial statements like income statements or balance sheets. However, I can still analyze its performance based on market data and blockchain metrics.

**Market Data**
As of the latest available data, DOT is trading at around $25.43 USD per token, with a market capitalization of approximately $24.75 billion USD.

**Blockchain Metrics**
According to Polkadot's blockchain explorer, the platform has:

1. **Total Supply**: 987,579

<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 25910 (\N{CJK UNIFIED IDEOGRAPH-6536}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')
<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 30436 (\N{CJK UNIFIED IDEOGRAPH-76E4}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')
<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 20729 (\N{CJK UNIFIED IDEOGRAPH-50F9}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')


...

API 回應：{
  "ticker": "SOL/USDT",
  "preference": "綜合分析",
  "investment_horizon": "長線",
  "other_requirements": "策略建議"
}
API 回應：基於提供的基本面資料，以下是我對 SOL/USDT 的分析結果：

**符號分析**
符號為 SOL-USD，表示此股票是 Solana（SOL）貨幣對美元（USD）的交易對。

**背景介紹**
Solana 是一種基於區塊鏈技術的加密貨幣，旨在提高交易速度和降低交易成本。Solana 網絡使用了一種稱為 proof of history（PoH）的共識算法，並且具有高吞吐量和低延遲的特點。

**財務面分析**
由於提供的資料僅包含符號，因此無法進行財務面的深入分析。Solana 作為一種加密貨幣，不具備傳統企業的財務報表，例如損益表、資產負債表等。

**市場面分析**
Solana 作為一種加密貨幣，其價值受到市場供應和需求的影響。根據 CoinMarketCap 的資料，Solana 目前的總市值約為 130 億美元，排名第 7 在全球加密貨幣市場中。

**技術面分析**
Solana 的技術指標表現尚可，根據 TradingView 的資料，其 50 天移動平均線位於 150 USD 左右，而 200 天移動平均線位於 120 USD 左右，表明 Solana 的長期趨勢為上漲。

**結論**
綜上所述，Solana 作為一種加密貨幣，具有高速交易和低延遲的特點，市場需求穩定，技術指標表現尚可。因此，我們對 Solana 的基本面評價為中立偏正面，預期其價值將保持穩定或上漲。然而，請注意，加密貨幣市場具有很高的波動性，投資者應該進行深入的研究和分析後再做出投資決定。
API 回應：根據提供的技術面資料，我們將進行綜合分析並給出策略建議。

首先，讓我們看一下移動平均線的趨勢。20 日簡單移動平均線（SMA_20）為 124.00，20 日指數移動平均線（EMA_20）為 129.31，20 日加權移動平均線（WMA_20）為 128.62。三條線都位於上漲趨勢，表明長期趨勢為上漲。

接下來，我們查看相對強弱指數（RSI_14）的值為 56.09，位於中間區域，表明股票未出現超買或超賣的情況。

 M

<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 25910 (\N{CJK UNIFIED IDEOGRAPH-6536}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')
<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 30436 (\N{CJK UNIFIED IDEOGRAPH-76E4}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')
<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 20729 (\N{CJK UNIFIED IDEOGRAPH-50F9}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')


API 回應：{
  "ticker": "SOL/USDT",
  "preference": "綜合分析",
  "investment_horizon": "長線",
  "other_requirements": "策略建議"
}
API 回應：以下是對 SOL/USDT 基本面資料的分析：

**符號資料**

 Symbol: SOL-USD

**初步分析**

根據提供的資料，這是 Solana（SOL） cryptocurrency 的交易資料，交易對為 SOL/USD。

**市場背景**

Solana 是一個區塊鏈平台，旨在提高區塊鏈的速度和可擴展性。_SOL 是該平台的原生代幣，用於支付交易費和參與平台治理。

**技术面分析**

由於未提供財務報表和其他非財務資料，因此本分析將集中在技术面。 SOL/USD 的技术面表現看似良好，最近的價格趨勢呈上升走勢，表明投資者對 Solana 平台的看好。

**結論**

根據提供的資料，我們可以初步評估 Solana（SOL）的基本面情況，看似良好。隨著區塊鏈技術的普及和 Solana 平台的發展前景，我們預期 SOL/USD 的價值走勢可能會繼續上漲。但是，因為未提供充分的財務報表和非財務資料，因此本分析結果僅供參考，投資前仍應進行全面性分析和風險評估。

如果需要進一步的分析，請提供更多的財務報表和非財務資料。
API 回應：根據提供的技術面資料，我們將進行綜合分析，給出結論和策略建議。

首先，讓我們分析 RSI_14 的值為 56.048303631725325，這表示 SOL/USDT 的短期價格趨勢處於中立區域，未顯示出明確的上漲或下跌趨勢。

接著，我們觀察 MACD 和 MACD_Signal 的值，MACD 為 2.0511817583565914，而 MACD_Signal 為 -0.5764438007061891，這意味著 MACD 線目前處於上漲趨勢，但 MACD_Signal 線處於下跌趨勢，兩者之間存在背离（divergence），這可能預示著短期內 SOL/USDT 的價格趨勢可能會出現變化。

 Moving Average 指標方面，SMA_20、EMA_20 和 WMA_20 的值分別為 124.00203590393066、129.30442353203

<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 25910 (\N{CJK UNIFIED IDEOGRAPH-6536}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')
<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 30436 (\N{CJK UNIFIED IDEOGRAPH-76E4}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')
<ipython-input-13-d9c9f769bc24>:28: UserWarning: Glyph 20729 (\N{CJK UNIFIED IDEOGRAPH-50F9}) missing from font(s) DejaVu Sans.
  plt.savefig('stock_plot.png')


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://247a9cf8a961b3afab.gradio.live
